Social Media:
I recently had to improve this code for a process at work and decided to highlight it on my GitHub account. 

Introduction
This could is simplifies the governence process for statistical reporting. Supervisors were having to conduct regular checks of employee reports. This took about one to two hours per day. A full-time admin assistant spent three to four days finding errors and making corrections per month. After implementing this script, supervisors are able to spend less than one hour per week and the administrative assistant tasks were almost completely resolved. The script interacts with Emergency Reporting NFIRS Module and JotForm API. 

In [1]:
import requests
import json
import smtplib
from hcfmosecrets import *
from email.message import EmailMessage

#I did not setup a test enviroment, so I am using a variable to control if the script is in test or production mode. If you want to test, set the variable to 'test'. If you want to run in production, set the variable to 'prod'
operationmode = 'test' 

#pull secret information



smtp_server = smtplib.SMTP(smtp_address)
tokenURL = 'https://login.microsoftonline.com/login.emergencyreporting.com/oauth2/v2.0/token'
allexposuresURL = 'https://data.emergencyreporting.com/agencyincidents/incidents/exposures'
getuserURL = 'https://data.emergencyreporting.com/agencyusers/users/'

summary = []
exposures = []
exposures_offset = 0

# get user information from Emergency Reporting for a specific userID
def getUser(userID):
    resp = requests.get('https://data.emergencyreporting.com/agencyusers/users/'+ userID,headers=requestheaders)
    response = resp.json()
    name = response["user"]["fullName"]
    email = response["user"]["email"]
    rtn = [name, email]
    return rtn

# get location data from Emergency Reporting for a specific exposureID
def getLocation(exposureID):
    resp = requests.get('https://data.emergencyreporting.com/agencyincidents/exposures/' + exposureID + '/location',
                        headers=requestheaders)
    response = resp.json()
    location = response["exposureLocation"]["milePostNumber"] + " " + response["exposureLocation"]["streetPrefix"] + " " + response["exposureLocation"]["streetName"] + " " + response["exposureLocation"]["streetType"] + " " + response["exposureLocation"]["streetSuffix"]
    return location

# get incident data from Emergency Reporting for a specific incidentID
def getincidentdata(incidentID): 
    resp = requests.get('https://data.emergencyreporting.com/agencyincidents/incidents/' + incidentID,
                        headers=requestheaders)
    response = resp.json()
    incidentData = [response["incident"]["incidentDateTime"],response["incident"]["incidentNumber"]]
    return incidentData    
    


## setup variables for token call from Emergency Reporting
tokenbody = {
    "grant_type":"client_credentials",
    "client_id": ER_client_id,
    "client_secret": ER_client_secret,
    "scope":"https://login.emergencyreporting.com/secure/.default offline_access"
}
tokenheaders = {
'content-type':'application/x-www-form-urlencoded'
}
tokenresponse = requests.post(tokenURL,data=tokenbody,headers=tokenheaders)

#place token and key for future use
tokenresponsejson = tokenresponse.json()
token = tokenresponsejson["access_token"]
key = ER_token

requestheaders = {
    "Authorization":token,
    "Ocp-Apim-Subscription-Key":key
}




In [2]:

#retrieve initial exposures, this only pulls ten exposures at a time
response = requests.get(allexposuresURL + "?filter=completedByUserID eq null", headers=requestheaders)
incon_exposures = response.json()
#print(incon_exposures["exposures"][0])

#Iterate through the list and filter out anything not investigations, then append exposures into exposures list
for each in incon_exposures['exposures']:
    if each["shiftsOrPlatoon"] == "Investigations":
        e = { 'exposureID': each["exposureID"], 'incidentID': each['incidentID'],  'assignedToUserID': each["assignedToUserID"], 'completedByUserID': each["completedByUserID"], 'incidentType': each["incidentType"] }
        exposures.append(e)

#if it returned more than 0, gather another ten exposures and append them to the list
while len(incon_exposures['exposures']) != 0:
    exposures_offset += 10
    print(exposures_offset)
    offsetURL = allexposuresURL+"?offset="+str(exposures_offset)+"&filter=completedByUserID eq null"
    response = requests.get(offsetURL, headers=requestheaders)
    incon_exposures = response.json()

    for each in incon_exposures['exposures']:
        if each["shiftsOrPlatoon"] == "Investigations":
            e = { 'exposureID': each["exposureID"], 'incidentID': each['incidentID'],  'assignedToUserID': each["assignedToUserID"], 'completedByUserID': each["completedByUserID"], 'incidentType': each["incidentType"] }
            exposures.append(e)


10


Temporarily disabling the section below, think it's a duplicate

In [3]:
# response = requests.get(allexposuresURL + "?filter=completedByUserID eq null", headers=requestheaders)
# incon_exposures = response.json()
# print(incon_exposures["exposures"][0])

{'exposureID': '70928769', 'incidentID': '71117320', 'shiftsOrPlatoon': 'Investigations', 'incidentType': '138', 'assignedToUserID': '152097', 'aidGivenOrReceived': 'N', 'hazmatReleased': 'N', 'primaryActionTaken': '86', 'secondaryActionTaken': None, 'thirdActionTaken': None, 'completedByUserID': None, 'reviewedByUserID': None, 'completedDateTime': None, 'reviewedDateTime': None, 'psapDateTime': '2023-01-05 12:14:34', 'dispatchNotifiedDateTime': None, 'initialResponderDateTime': None, 'hasPropertyLoss': '1', 'propertyLossAmount': None, 'hasContentLoss': '1', 'contentLossAmount': None, 'hasPreIncidentPropertyValue': '1', 'preIncidentPropertyValueAmount': None, 'hasPreIncidentContentsValue': '1', 'preIncidentContentsValueAmount': None, 'complaintReportedByDispatch': None, 'rowVersion': '0000000A811F8FD1'}


In [3]:
## come back and add additional code to grab all incomplete, loop and offset  
for each in exposures:
    if each["assignedToUserID"] is None:
        print("No user assigned to this incident", each["incidentID"])
    else:
        user = getUser(each["assignedToUserID"])
        print(user)
        location = getLocation(each["exposureID"])
        incidentdata = getincidentdata(each["incidentID"])
    
        msgContent = "You have an incomplete incident assigned to you in Emergency Reporting. See incident 2022-" + incidentdata[1] + " for a NFIRS " + each["incidentType"] + " type call, located at " + location + ". This incident occured on " + incidentdata[0] + ". \n Please complete this report during your next shift. Thank you."
        message = EmailMessage()
        message.set_content(msgContent)
        message['Subject'] = 'Incomplete ER 2022-' + incidentdata[1]
        message['From'] = 'arsonsup@fmo.hctx.net'
        if operationmode == 'prod':
            message['To'] = user[1]
        else:
            message['To'] = 'james.singleton@fmo.hctx.net'
        
        smtp_server.send_message(message)
        print("message sent to ", user[1])
        summary.append({"Investigator": user[1], "incident": incidentdata[1] })



In [4]:
msgContent = "The automated system has sent reminders to \n"
msgContent += "Investigator \tIncident Number \n"

for r in summary:
    msgContent += r["Investigator"]
    msgContent += "\t"
    msgContent += "2022-" + r["incident"]
    msgContent += "\n"

message = EmailMessage()
message.set_content(msgContent)
message['Subject'] = 'Incomplete ER Reports'
message['From'] = 'arsonsup@fmo.hctx.net'
if operationmode == 'prod':
    message['Cc'] = ['tonya.spencer@fmo.hctx.net','kyle.pulley@fmo.hctx.net']
message['To'] = 'james.singleton@fmo.hctx.net'
      
smtp_server.send_message(message)
print(msgContent)




The automated system has sent reminders to 
Investigator 	Incident Number 



In [5]:
### reset variables
exposures = []
exposures_offset = 0
### QC reports already submitted


#retrieve initial exposures
response = requests.get(allexposuresURL + "?filter=reviewedByUserID eq null", headers=requestheaders)
incon_exposures = response.json()
print(incon_exposures["exposures"][8])

#append exposures into exposures list
for each in incon_exposures['exposures']:
    if each["shiftsOrPlatoon"] == "Investigations":
        e = { 'exposureID': each["exposureID"], 'incidentID': each['incidentID'],  'assignedToUserID': each["assignedToUserID"], 'completedByUserID': each["completedByUserID"], 'incidentType': each["incidentType"], 'completedDateTime': each["completedDateTime"] }
        exposures.append(e)

#if more than 10 exposures, then gather next set
while len(incon_exposures['exposures']) != 0:
    exposures_offset += 10
    print(exposures_offset)
    offsetURL = allexposuresURL+"?offset="+str(exposures_offset)+"&filter=reviewedByUserID eq null"
    response = requests.get(offsetURL, headers=requestheaders)
    incon_exposures = response.json()

    for each in incon_exposures['exposures']:
        if each["shiftsOrPlatoon"] == "Investigations":
            e = { 'exposureID': each["exposureID"], 'incidentID': each['incidentID'],  'assignedToUserID': each["assignedToUserID"], 'completedByUserID': each["completedByUserID"], 'incidentType': each["incidentType"], 'completedDateTime': each["completedDateTime"] }
            exposures.append(e)



IndexError: list index out of range

In [6]:
print(exposures[1])

IndexError: list index out of range

In [22]:
incident = getincidentdata("71066301")
print(incident)

['2023-01-03 03:59:47.000', '000316']


In [ ]:
smtp_server.quit()
exit()